In [1]:
'''
Obtiene letra de canciones
'''
import cancion_logica, letras_logica

# obtener canciones pendientes si letras registradas
canciones_pendientes = cancion_logica.obtenerCancionesPendientes()

for cancion_pendiente in canciones_pendientes:
   # obtiene el nombre de la canción y del cantante de la base de datos que no tiene letras
   nombre_cancion = cancion_pendiente['cancion']
   nombre_cantante = cancion_pendiente['cantante']
   
   # obtiene letra de canción desde Lyrics
   letra_cancion = letras_logica.obtenerLetraConLyrics(nombre_cancion, nombre_cantante)
   
   # actualiza la letra en la colección de la canción
   resultado_final = cancion_logica.actualizarCancionPendiente(cancion_pendiente, letra_cancion)
   print(resultado_final)

URL búsqueda en Lyrics:  https://www.lyrics.com/lyrics/PELELE%20MORAD
cancion lyrics:  Y.N.M.S.
autor lyrics:  
url letra lyrics:  location.href='https://www.lyrics.com/lyric-lf/7637049/7aared/Y.N.M.S.';
cancion bdd: PELELE
autor bdd: MORAD
-----------------
cancion lyrics:  Pelele
autor lyrics:  Morad
url letra lyrics:  location.href='https://www.lyrics.com/lyric-lf/6852059/Morad/Pelele';
cancion bdd: PELELE
autor bdd: MORAD
-----------------
location.href='https://www.lyrics.com/lyric-lf/6852059/Morad/Pelele';
cancion lyrics:  2x1
autor lyrics:  Morad
url letra lyrics:  location.href='https://www.lyrics.com/lyric-lf/2251955/Morad/2x1';
cancion bdd: PELELE
autor bdd: MORAD
-----------------
referencia url letra canción:  location.href='https://www.lyrics.com/lyric-lf/6852059/Morad/Pelele';
url letra canción:  https://www.lyrics.com/lyric-lf/6852059/Morad/Pelele;
documento actualizado correctamente
